<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/busqueda_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se buscara un numero dentro de un vector

EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y buscar un numero dentro de un vector. El ejemplo es ilustrativo para entender los multi hilos de la GPU de una dimenciones.

---
# 2 Armado del ambiente
Toma la direcciòn web de una imagen con  acceso público en internet, la deja disponible al contexto de ejecuciòn del cuaderno colab.

In [87]:
#@title # 2.1 Parámetros de ejecución
#@markdown ### Especifique las dimeciones de la matrices:

try: 
  cant_element =  500001#@param {type:"integer"}
  encontrar_numero =  9#@param {type:"integer"}

except:
  print("Error de ingresar los parametros")








Validar las matrices

In [23]:

if cant_element < 0 :
  print("ERROR LA CANTIDAD DE ELEMENTOS NO PUEDE SER MENOR QUE CERO")

---
## 2.2 Instala en el cuaderno el módulo CUDA de Python.

In [30]:
!pip install pycuda

---
# 3 Desarrollo
Ejecución del algoritmo escala de grises en GPU.

In [89]:
%matplotlib inline
from datetime import datetime
tiempo_total = datetime.now()

import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from   pycuda.compiler import SourceModule

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

tiempo_definicion_vector = datetime.now()

# CPU - Defino la memoria del vector de n elementos en cpu.
vector = numpy.array(range(-cant_element, cant_element, 2))
vector = vector.astype(numpy.int32())

# CPU - Defino la memoria del vector position_v de 1 elemento en cpu.
position_v    = numpy.random.randn(1)*0
position_v[0] = -1
position_v    = position_v.astype(numpy.int32())

tiempo_definicion_vector = datetime.now() - tiempo_definicion_vector

tiempo_find_secuencial = datetime.now()

position = -1
acu_sec = 0
for i in range(cant_element):
  if vector[i] == encontrar_numero:
    position = i
    break

tiempo_find_secuencial = datetime.now() - tiempo_find_secuencial
    

tiempo_reserva_memoria_GPU = datetime.now()
# CPU - reservo la memoria GPU.
vector_Gpu = cuda.mem_alloc(vector.nbytes)
position_v_Gpu = cuda.mem_alloc(position_v.nbytes)
tiempo_reserva_memoria_GPU = datetime.now() - tiempo_reserva_memoria_GPU

tiempo_copia_memoria_GPU = datetime.now()
# GPU - Copio la memoria al GPU.
cuda.memcpy_htod(vector_Gpu, vector)
cuda.memcpy_htod(position_v_Gpu, position_v)
tiempo_copia_memoria_GPU = datetime.now() - tiempo_copia_memoria_GPU

#CPU - Defino la funcion kernel que ejecutará en GPU
module = SourceModule("""
__global__ void find(int tam, int number_find, int *vector, int *position)
{
    // Calculo las coordenadas del Thread en dos dimensiones.
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    
    // Verifico que los Thread, esten dentro de las dimensiones del vector
    if( idx < tam ) {
      if (vector[idx] == number_find) {
        position[0] = idx;
      }
    }
}

""")

# CPU - Genero la función kernel.
kernel = module.get_function("find")

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cant_element+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

tiempo_find_paralelo = datetime.now()

kernel( numpy.int32(cant_element), 
        numpy.int32(encontrar_numero), 
        vector_Gpu, 
        position_v_Gpu, 
        block=( dim_hilo, 1, 1 ),
        grid=(dim_bloque, 1,1) )

tiempo_find_paralelo = datetime.now() - tiempo_find_paralelo

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( position_v, position_v_Gpu )
tiempo_total = datetime.now() - tiempo_total

print("Cantidad de elementos del vector: ", cant_element)
print("Numero a encontrar: ", encontrar_numero)
print("Vector: \n", vector)
if position == -1 or position_v[0] == -1:
  print("No se encontro el numero: ", encontrar_numero)
else:
  print("Se encontro el numero: ", encontrar_numero)
  print("Posicion dentro del vector es: ", position_v[0])
  print("Posicion dentro del vector es: ", position)

print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU  : ", tiempo_en_ms( tiempo_find_paralelo ), "[ms]" )
print("Tiempo CPU  : ", tiempo_en_ms( tiempo_find_secuencial ), "[ms]" )

Thread x:  256 , Bloque x: 1954
Cantidad de elementos del vector:  500001
Numero a encontrar:  9
Vector: 
 [-500001 -499999 -499997 ...  499995  499997  499999]
Se encontro el numero:  9
Posicion dentro del vector es:  250005
Posicion dentro del vector es:  250005
Tiempo TOTAL:  697.083 [ms]
Tiempo GPU  :  0.246 [ms]
Tiempo CPU  :  354.693 [ms]


---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random(n)       | Crear una vector con numeros ramdoms de n elementos.
CPU      |  ar.astype(np.int32()) | Convierte los elemementos del array ar en elementos de tipo int 32 bit.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las imagenes en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las imagenes al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x            | Calcula las dimensiones para la ejecuciòn de 1D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa los atributos de la imagen.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.



---
# 5 Conclusiones

Las conclusiones son explicadas en clase...

---
# 6 Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[4] 2009, SINTESIS DIGITAL DE COLOR UTILIZANDO TONOS DE GRIS, ING. JESÚS GUSTAVO FLORES ERAÑA : [PDF](https://ninive.uaslp.mx/xmlui/bitstream/handle/i/2264/MCA1SDC00901.pdf?sequence=1&isAllowed=y)
